# CS 583 Project 1

### Fangda Fan, Xiaohan Liu

- Implement: MS-Apriori (excluding rule generation)
- Consider: multiple minimum supports, support difference constraint, and item constraints
- Item constraints: Two types
    - Cannot–be-together: sets of items cannot be in the same itemsets (pairwise), 
        - e.g., {1, 2, 3} and {6, 7, 9, 10}
    - Must-have: every itemset must have, 
        - e.g., (1 or 2)
- Deadline: Feb 9, 2017 

In [1]:
import numpy as np
import scipy.sparse as sp
import pandas as pd

## Read Data and Arguments

### Read Arguments

In [2]:
def input_args(file_args):
    ms = dict()
    sdc = 1
    x_cannot = []
    x_must = []
    for i in open(file_args, "r"):
        i = i.rstrip("\n")
        if i.startswith("MIS"):
            j = i.split(" = ")
            ms.update({j[0][4:-1]: float(j[1])})
        elif i.startswith("SDC"):
            sdc = float(i.split("=")[1])
        elif i.startswith("cannot_be_together"):
            x_cannot = [j.split(", ") for j in i.split(": ")[1][1:-1].split("}, {")]
        elif i.startswith("must"):
            x_must = [j for j in i.split(": ")[1].split(" or ")]
    op = {"ms": ms, "sdc": sdc, "x_cannot": x_cannot, "x_must": x_must}
    return(op)

In [3]:
file_args = "proj1_parameter-file.txt"
file_data = "proj1_input-data.txt"
#file_args = "proj1_ex13_args.txt"
#file_data = "proj1_ex13_data.txt"

args = input_args(file_args)
args

{'ms': {'10': 0.43,
  '100': 0.1,
  '120': 0.2,
  '140': 0.15,
  '20': 0.3,
  '30': 0.3,
  '40': 0.4,
  '50': 0.4,
  '60': 0.3,
  '70': 0.2,
  '80': 0.2,
  '90': 0.2},
 'sdc': 0.1,
 'x_cannot': [['20', '40'], ['70', '80']],
 'x_must': ['20', '40', '50']}

In [4]:
ms = pd.Series(args["ms"], name = "MIS").sort_values().reset_index()
id_dict = ms["index"].to_dict()
ms_dict = ms["MIS"].to_dict()
id_dict_inv = {val: key for key, val in id_dict.items()}
x_must = [id_dict_inv[i] for i in args["x_must"]]
x_cannot = [tuple(np.sort([id_dict_inv[j] for j in i])) for i in args["x_cannot"]]
print("Index-item dictionary:", id_dict)
print("MIS index", ms_dict)
print("Cannot-be-Together index:",  x_cannot)
print("Must-have:", x_must)

Index-item dictionary: {0: '100', 1: '140', 2: '120', 3: '70', 4: '80', 5: '90', 6: '20', 7: '30', 8: '60', 9: '40', 10: '50', 11: '10'}
MIS index {0: 0.10000000000000001, 1: 0.14999999999999999, 2: 0.20000000000000001, 3: 0.20000000000000001, 4: 0.20000000000000001, 5: 0.20000000000000001, 6: 0.29999999999999999, 7: 0.29999999999999999, 8: 0.29999999999999999, 9: 0.40000000000000002, 10: 0.40000000000000002, 11: 0.42999999999999999}
Cannot-be-Together index: [(6, 9), (3, 4)]
Must-have: [6, 9, 10]


### Read Transaction Data

In [5]:
def input_data(file_data, columns):
    s = pd.read_csv(file_data, header = None, sep = "\t",squeeze = True)
    op = s.str[1:-1].str.get_dummies(sep = ", ").reindex(columns = columns, fill_value = 0)
    return(op)

In [6]:
da = input_data(file_data, ms["index"])
X = da.values
da

index,100,140,120,70,80,90,20,30,60,40,50,10
0,0,0,0,1,1,1,1,1,0,0,1,0
1,0,0,0,1,1,0,1,0,0,0,0,1
2,0,0,0,0,1,0,1,0,0,0,0,1
3,0,0,0,0,1,0,1,1,0,0,0,0
4,0,0,0,0,1,0,1,0,0,0,0,0
5,1,1,1,1,1,1,1,1,0,0,1,0


In [7]:
def sup(xL):
    op = np.mean([X[:, i].all(axis = 1) for i in xL], axis = 1)
    return(op)

In [8]:
I = [(i,) for i, ival in enumerate(ms_dict)]
Isup = sup(I)
sup_dict = dict(zip(I, Isup))
I, sup_dict

([(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,)],
 {(0,): 0.16666666666666666,
  (1,): 0.16666666666666666,
  (2,): 0.16666666666666666,
  (3,): 0.5,
  (4,): 1.0,
  (5,): 0.33333333333333331,
  (6,): 1.0,
  (7,): 0.5,
  (8,): 0.0,
  (9,): 0.0,
  (10,): 0.33333333333333331,
  (11,): 0.33333333333333331})

## 2. Candidate Generation
### Level 1

In [9]:
Li = (Isup > ms["MIS"]).argmax()
L = [i for i in range(Li, ms.shape[0]) if Isup[i] > ms["MIS"][Li]]
F = [[(i,) for i in np.where(Isup > ms["MIS"])[0]]]
Li, L, F

(0,
 [0, 1, 2, 3, 4, 5, 6, 7, 10, 11],
 [[(0,), (1,), (3,), (4,), (5,), (6,), (7,)]])

### Level $\geq$ 2

In [10]:
def pair_sup_mis(x):
    x_t = x[:, np.newaxis]
    x_sup = sup(x_t)
    x_mis = [ms_dict[i] for i in x]
    x_sup_t = x_sup[:, np.newaxis]
    iL = sp.coo_matrix(np.triu((x_sup_t >= x_mis).T & (np.abs(x_sup_t - x_sup) < args["sdc"]), 1)).nonzero()
    op = list(zip(x[iL[0]], x[iL[1]]))
    return(op)
def frequent(xL):
    x_sup = sup(xL)
    sup_dict.update(dict(zip(xL, x_sup)))
    op = [xL[j] for j in np.where(x_sup >= [ms_dict[i[0]] for i in xL])[0]]
    xL_dropfirst = set(tuple(i[1:]) for i in xL)
    sup_dict.update(dict(zip(xL_dropfirst, sup(xL_dropfirst))))
    return(op)
def append_set(xL, x_base):
    if len(xL):
        op = [tuple(i) for i in np.hstack([np.tile(x_base, (len(xL), 1)), xL])]
    else:
        op = []
    return(op)
def prune_candidate(xL):
    if xL:
        op = [xL[l] for l in np.where(np.all([[any(set(k).issubset(j) for k in F[-1]) for j in np.delete(xL, i, axis = 1)] for i in range(1, len(xL[0]))], axis = 0))[0]]
    else:
        op = []
    return(op)

In [11]:
C = [i for i in pair_sup_mis(np.array(L)) if i[0] in np.array(F[0]).T[0]]
C

[(0, 1), (0, 2), (1, 2), (3, 7), (4, 6), (5, 10), (5, 11)]

In [12]:
while C:
    F.append(frequent(C))
    Ls = pd.DataFrame(F[-1])
    C = sum([append_set(pair_sup_mis(group.values), name) for name, group in Ls.groupby(list(range(len(F)-1)))[len(F)-1]], [])
    C = prune_candidate(C)
F

[[(0,), (1,), (3,), (4,), (5,), (6,), (7,)],
 [(0, 1), (0, 2), (1, 2), (3, 7), (4, 6), (5, 10)],
 [(0, 1, 2)]]

### Prune with Item Constraints

In [13]:
F_prune = [[j for j in i if any(k in j for k in x_must) & ~any(set(k).issubset(j) for k in x_cannot)] for i in F]
F_prune

[[(6,)], [(4, 6), (5, 10)], []]

### Item Supports for Association Rule

In [14]:
sup_dict

{(0,): 0.16666666666666666,
 (0, 1): 0.16666666666666666,
 (0, 1, 2): 0.16666666666666666,
 (0, 2): 0.16666666666666666,
 (1,): 0.16666666666666666,
 (1, 2): 0.16666666666666666,
 (2,): 0.16666666666666666,
 (3,): 0.5,
 (3, 7): 0.33333333333333331,
 (4,): 1.0,
 (4, 6): 1.0,
 (5,): 0.33333333333333331,
 (5, 10): 0.33333333333333331,
 (5, 11): 0.0,
 (6,): 1.0,
 (7,): 0.5,
 (8,): 0.0,
 (9,): 0.0,
 (10,): 0.33333333333333331,
 (11,): 0.33333333333333331}

## Output

In [15]:
def output_frequent(F):
    op = []
    for i, ival in enumerate(F):
        if ival:
            op.append("Frequent {}-itemsets\n".format(i+1))
            if i == 0:
                op += ["\t{} : {}".format(int(sup_dict[j]*len(da)), {int(id_dict[k]) for k in j}) for j in ival]
            else:
                op += ["\t{} : {}\nTailcount = {}".format(int(sup_dict[j]*len(da)), {int(id_dict[k]) for k in j}, int(sup_dict[j[1:]]*len(da))) for j in ival]
            op.append("\nTotal number of frequent {}-itemsets = {}\n\n".format(i+1, len(ival)))
    print("\n".join(op))
output_frequent(F_prune)

Frequent 1-itemsets

	6 : {20}

Total number of frequent 1-itemsets = 1


Frequent 2-itemsets

	6 : {80, 20}
Tailcount = 6
	2 : {90, 50}
Tailcount = 2

Total number of frequent 2-itemsets = 2


